In [9]:
import csv
import json

import numpy as np
import pandas

ranking_path = "/home/sasce/PycharmProjects/GitHubClassificationDataset/data/pairrank/mean_std_ranking.csv"
ranking = pandas.read_csv(ranking_path)
ranking = ranking.sort_values('mean', ascending=False)
ranking['description'] = ranking['description'].fillna('')
cluster_feature = 'mean'

In [10]:
def order_clusters(cluster):
    seen = {}
    n = len(set(cluster))
    res = []
    for i in cluster:
        if i not in seen:
            seen[i] = len(seen) + 1
        res.append(seen[i])

    return res

In [16]:
from sklearn.cluster import KMeans
data = np.array(ranking['mean'].to_list())
kmeans = KMeans(n_clusters=8).fit(data.reshape(-1,1))
clusters = kmeans.predict(data.reshape(-1,1))
ranking['cluster'] =  order_clusters(clusters)
ranking['pos'] = list(range(len(ranking['mean'])))
df = ranking[['topic', 'mean', 'pos', 'cluster']].copy(deep=True)
df.to_csv('scatter.csv', index=False)


In [17]:
freq_path = "/home/sasce/PycharmProjects/GitHubClassificationDataset/data/annotated_github_topics_wikidata.json"
freqs = {}

with open(freq_path, 'rt') as inf:
    for line in inf:
        a = json.loads(line)
        freqs[a["Wikidata Title"]] = a['Frequency']


In [18]:
from collections import defaultdict

topics = ranking['topic']
freq = [freqs[t] for t in topics]
clusters = ranking['cluster']

clusters_freq = defaultdict(lambda: 0)
clusters_freq_u = defaultdict(lambda: 0)
for c, f in zip(clusters, freq):
    clusters_freq[c] += f
    clusters_freq_u[c] += 1


with open('cluster_distr.csv', 'wt') as outf, open('cluster_distr_u.csv', 'wt') as outfu:
    writer = csv.writer(outf)
    writer2 = csv.writer(outfu)
    writer.writerow(['cluster', 'freq'])
    writer2.writerow(['cluster', 'freq'])
    for c in clusters_freq:
        writer.writerow([c, clusters_freq[c]])
        writer2.writerow([c, clusters_freq_u[c]])

In [19]:
freq_ord = [freqs[t] for t in df['topic']]
df['freq'] = freq_ord
df.corr('pearson')

,mean,pos,cluster,freq
mean,1.000000,-0.960734,-0.974148,0.110858
pos,-0.960734,1.000000,0.966942,-0.134135
cluster,-0.974148,0.966942,1.000000,-0.128819
freq,0.110858,-0.134135,-0.128819,1.000000


In [ ]:
import plotly.express as px

fig = px.scatter(ranking, y = 'mean', x='pos', color = 'cluster')
fig.show()


In [ ]:
means = ranking['mean']

diff = [abs(a-b) for a, b in zip(means, means[1:])]

In [ ]:
fig = px.line(x = range(len(diff)), y=diff)
fig.show()
